In [1]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
from quantfreedom.utils.caching import clear_cache
from quantfreedom.backtester.base.base import plot_trades_all_info, rsi_above_entries
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
pd.options.display.float_format = '{:,.2f}'.format
# clear_cache()

# qf.CCXTData.data_download(
#     exchange='bybit',
#     start='2021-11-10T00:00:00Z',
#     end='2022-01-23T00:00:00Z',
#     symbol='BTCUSDT',
#     timeframe='30m',
# ).to_csv('30minshort.csv')

prices = pd.read_csv('30minshort.csv', index_col='open_time')
entries = rsi_above_entries(
    timeperiods=[20],
    above_ranges=[50],
    prices=prices.close
)


In [2]:
final_array, order_records = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.ShortEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    gains_pct_filter=40,
    sl_pcts=[2,3,4],
    # tsl_true_or_false=True,
    # tsl_pcts_init=np.arange(2,5,1),
    # tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    # tsl_trail_by_pct=np.arange(1,4,1),
    # tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)

In [3]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(10)
    

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
0,0,0,27,471.43,"66,420.50","66,420.50",NaN,ShortEntry,NaN,"1,000.00","67,748.91",NaN,"62,116.64"
1,1,0,27,471.43,"67,748.91","66,420.50",0.57,ShortSL,-10.00,990.00,"67,748.91",NaN,"62,116.64"
2,2,0,28,466.72,"68,028.50","68,028.50",NaN,ShortEntry,NaN,990.00,"69,389.07",NaN,"63,620.45"
3,3,0,29,466.72,"68,391.00","68,209.27",NaN,ShortEntry,NaN,990.00,"69,573.45",NaN,"63,789.50"
4,4,0,30,466.72,"68,450.00","68,289.32",NaN,ShortEntry,NaN,990.00,"69,655.11",NaN,"63,864.37"
5,5,0,31,466.72,"68,325.00","68,298.24",NaN,ShortEntry,NaN,990.00,"69,664.20",NaN,"63,872.71"
6,6,0,42,"1,866.87","63,872.71","68,298.24",2.17,ShortTP,118.80,"1,108.80","69,664.20",NaN,"63,872.71"
7,7,0,92,522.72,"65,069.00","65,069.00",NaN,ShortEntry,NaN,"1,108.80","66,370.38",NaN,"60,852.72"
8,8,0,93,522.72,"65,254.50","65,161.62",NaN,ShortEntry,NaN,"1,108.80","66,464.85",NaN,"60,939.33"
9,9,0,99,522.72,"64,614.00","64,978.05",NaN,ShortEntry,NaN,"1,108.80","66,277.61",NaN,"60,767.66"


In [7]:
df_cart = pd.DataFrame(final_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2
total_trades,44.00,8.00,3.00
gains_pct,48.09,76.12,47.61
win_rate,29.55,50.00,66.67
to_the_upside,0.23,0.76,0.60
total_pnl,480.87,761.20,476.10
ending_eq,"1,480.87","1,761.21","1,476.10"
settings_id,0.00,1.00,2.00
max_equity_risk_pct,4.00,4.00,4.00
risk_rewards,3.00,5.00,6.00
size_pct,1.00,1.00,1.00


In [8]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[0:20],
)
